In [1]:
from openai import OpenAI
import json
from typing import Dict, List, Optional, Any
from fewshots import analyser_fs, planner_fs
import os


In [ ]:
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

In [9]:
class DeploymentSystem:
    def __init__(self):
        self.specs: Dict[str, Dict[str, Any]] = {}
        self.summary: str = ""
        self.chat_history: List[str] = []
        self.current_prompt: Optional[str] = None
        self.plan_history: List[Dict[str, Any]] = []
        self.analyzer_fs = analyser_fs
        self.planner_fs = planner_fs
        # self.optimizer_fs = optimizer_fs
        # self.validator_fs = validator_fs
        # self.summarizer_fs = summarizer_fs
    
    def llm(self, prompt: str) -> str:
        """Call the LLM with the given prompt and return the response."""
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt
        )
        return response.output_text
    
    # Agent 1: Summarizer
    def summarizer(self) -> str:
        """Summarize the current state of the system (devices, plan, etc.)."""
        pass

    # Agent 2: Analyzer
    def analyzer(self, user_prompt: str) -> str:
        """Analyze and expand the user's intent from the input prompt and provide a prompt for the planner"""
        self.current_prompt = user_prompt
        # Use few-shot examples to guide the analysis
        prompt_header = ("You are an AI system that converts user deployment requests into structured JSON intent to help an LLM planner generate deployment plans. Here are some examples:\n\n")
        few_shot_blocks = []
        for example in self.analyzer_fs:
            formatted=f"User: {example['prompt']}\nIntent: {json.dumps(example['output'], indent=2)}\n"
            few_shot_blocks.append(formatted)
        few_shot_text = "\n\n".join(few_shot_blocks)
        full_prompt = f"{prompt_header}{few_shot_text}\n\nUser: {user_prompt}\nIntent:"
        self.chat_history.append(f"User:",user_prompt)
        intent = self.llm(full_prompt)
        self.chat_history.append(f"AI:", intent)
        return intent


    # Agent 3: Planner
    def planner(self, intent: str) -> Dict[str, Any]:
        """Generate a potential plan for device deployment."""
        prompt_header = (
        "You are an LLM planner that creates deployment plans for IoT–Edge–Cloud systems.\n"
        "You will be given:\n"
        "1. The current system specifications (devices, edges, clouds)\n"
        "2. The user's high-level intent (goals, constraints, preferences)\n"
        "3. The last deployment plan (if any)\n"
        "4. The original user prompt\n"
        "Your job is to provide a JSON plan along with a short 'Chain of Thought' reasoning.\n\n"
        "Here are some examples:\n\n"
    )
        few_shot_blocks = []
        for example in self.planner_fs:
            formatted = (
                f"Intent:\n{json.dumps(example['intent'], indent=2)}\n"
                f"Specs:\n{json.dumps(example['specs'], indent=2)}\n"
                f"Last Plan:\n{json.dumps(example.get('last_plan', {}), indent=2)}\n"
                f"Output:\n{json.dumps(example['output'], indent=2)}\n"
            )
            few_shot_blocks.append(formatted)
        few_shot_text = "\n\n".join(few_shot_blocks)
        full_prompt = (
            f"{prompt_header}{few_shot_text}\n\n"
            f"Now generate a new plan.\n"
            f"User Prompt: {self.current_prompt}\n"
            f"Intent:\n{intent}\n"
            f"Specs:\n{json.dumps(self.specs, indent=2)}\n"
            f"Last Plan:\n{json.dumps(self.plan_history[-1] if self.plan_history else {}, indent=2)}\n"
            f"Output:"
    )
        self.chat_history.append(("Planner:", self.current_prompt))
        plan = self.llm(full_prompt)
        self.chat_history.append(f"AI:", plan)
        return plan # Assuming the plan is a JSON string, i am not checking its validity here
    # Agent 4: Optimizer
    def optimizer(self, plan: Dict[str, Any]) -> Dict[str, Any]:
        """Optimize the deployment plan based on latency, bandwidth, etc."""
        # For simplicity, we will just return the plan as is.
        # In a real implementation, this would involve more complex logic.
        # Before that, we will make sure the the plan is a dictory object.
        if isinstance(plan, str):
            try:
                plan = json.loads(plan)
            except json.JSONDecodeError:
                print("Error decoding plan JSON.")
                return {}
        return plan["plan"] if "plan" in plan else plan
    # Agent 5: Resource Validator
    def validate_resources(self, plan: Dict[str, Any]) -> bool:
        """Check if the plan meets all constraints (CPU, RAM, bandwidth)."""
        pass

    # Agent 6: Goal Validator
    def validate_goals(self, plan: Dict[str, Any]) -> bool:
        """Check if the plan meets the user's goals."""
        pass


    # Utility: Generate a final plan end-to-end
    def generate_plan(self, user_prompt: str):
        """Run all agents sequentially to create the final plan."""
        while True:
            analysis = self.analyzer(user_prompt)
            plan = self.planner(analysis)
            optimized_plan = self.optimizer(plan)
            self.plan_history.append(optimized_plan)
            # Validate resources
            while not self.validate_resorces(optimized_plan):
                print("❌ Resource constraints validation failed.")
                retry = input("Would you like to retry the optimization, replan all, or exit? (yes/replan/exit): ").lower()
                if retry == 'yes':
                    print("Retrying optimization...")
                    optimized_plan = self.optimizer(plan)
                elif retry == 'replan':
                    print("Retrying planning from start...")
                    break  # Break inner loop to re-enter outer loop and restart planning
                else:
                    print("Exiting deployment generation.")
                    return
            else:
                # Only proceed to goal validation if resource validation succeeded
                while not self.validate_goals(optimized_plan):
                    print("❌ Goal validation failed.")
                    retry = input("Would you like to retry the optimization, replan all, or exit? (yes/replan/exit): ").lower()
                    if retry == 'yes':
                        print("Retrying optimization...")
                        optimized_plan = self.optimizer(plan)
                    elif retry == 'replan':
                        print("Retrying planning from start...")
                        break  # Break inner loop to re-enter outer loop and restart planning
                    else:
                        print("Exiting deployment generation.")
                        return
                else:
                    print("✅ Plan successfully validated.")
                    return optimized_plan


In [10]:
templan = planner_fs[0]["output"]
DeploymentSystem = DeploymentSystem()
plans = DeploymentSystem.optimizer(plan=templan)

In [11]:
print(plans)  # Assuming plans is a dictionary, you can print it directly

{'IoT_Sensors_A': {'type': 'IoT', 'cpu': 0.2, 'ram': 0.1, 'bandwidth_to_edge': 0.5, 'power': 1.5, 'count': 25, 'connected_to': 'Edge_Nodes_A', 'latency': {'Edge_Nodes_A': 5, 'Cloud_Node_1': 50, 'IoT_Sensors_A': 0}}, 'IoT_Sensors_B': {'type': 'IoT', 'cpu': 0.2, 'ram': 0.1, 'bandwidth_to_edge': 0.5, 'power': 1.5, 'count': 25, 'connected_to': 'Edge_Nodes_B', 'latency': {'Edge_Nodes_A': 5, 'Edge_Nodes_B': 5, 'Cloud_Node_1': 50, 'IoT_Sensors_A': 0}}, 'Edge_Node_A': {'type': 'Edge', 'cpu_capacity': 8, 'ram_capacity': 16, 'bandwidth_capacity': 20, 'bandwidth_to_cloud': 5, 'connected_iot': ['IoT_Sensors_A'], 'latency': {'Edge_Node_A': 0, 'Edge_Node_B': 5, 'Cloud_Node_1': 50, 'IoT_Sensor_A': 5, 'IoT_Sensor_B': 5}}, 'Edge_Node_B': {'type': 'Edge', 'cpu_capacity': 8, 'ram_capacity': 16, 'bandwidth_capacity': 20, 'bandwidth_to_cloud': 5, 'connected_iot': ['IoT_Sensors_B'], 'latency': {'Edge_Node_A': 5, 'Edge_Node_B': 0, 'Cloud_Node_1': 50, 'IoT_Sensor_A': 5, 'IoT_Sensor_B': 5}}, 'Cloud_Node': {'ty